In [ ]:
from dataset import load_morph_data_npz
from age_gender_model import age_gender_mobile_net
from keras.optimizers import Adam
import keras
import os

In [ ]:
data_path = '/src/dataset/morph2/morph2_db_align.npz'
(x_train, age_train, gender_train), (x_test, age_test, gender_test), image_shape0 = load_morph_data_npz(data_path)

In [ ]:
model = age_gender_mobile_net((image_shape0, image_shape0, 3))
model.compile(optimizer=Adam(), loss=["mae", "mae"], 
              metrics={'pred_age': 'mae', 'pred_gender': 'mae'}, 
              loss_weights=[0.5, 1])

In [ ]:
tensor_board = keras.callbacks.TensorBoard(log_dir='/src/tensorboard', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

if not os.path.exists('age_gender_mobile_net_check_point'):
    os.mkdir('age_gender_mobile_net_check_point')
    
model_check_point = keras.callbacks.ModelCheckpoint("age_gender_mobile_net_check_point/weights.{epoch:02d}-{val_loss:.2f}.hdf5",
                                                    monitor="val_loss",
                                                    verbose=1,
                                                    save_best_only=True,
                                                    mode="auto")

In [ ]:
print('Training ------------')
model.fit(x_train, [age_train, gender_train], batch_size=256, epochs=300,
          callbacks=[tensor_board, model_check_point],
          validation_data=(x_test, [age_test, gender_test]))